In [1]:
!pip install fastapi uvicorn sqlalchemy pyngrok nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00


In [2]:
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import RedirectResponse
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import string
import random

# Allow nested event loops for Colab
nest_asyncio.apply()

# Create FastAPI app
app = FastAPI()

# SQLite for simplicity in Colab (replace with PostgreSQL URI in production)
DATABASE_URL = "sqlite:///./shortener.db"

# Setup SQLAlchemy
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

<ipython-input-2-b48068becc2d>:25: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# Database model for storing URLs
class URL(Base):
    __tablename__ = "urls"

    id = Column(Integer, primary_key=True, index=True)
    original_url = Column(String, nullable=False)
    short_code = Column(String, unique=True, index=True)

# Create the database table
Base.metadata.create_all(bind=engine)

In [4]:
# Schema for request validation
class URLBase(BaseModel):
    original_url: str

In [8]:
!wget -q -c https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok-v3-stable-linux-amd64.zip
!chmod +x ngrok

Archive:  ngrok-v3-stable-linux-amd64.zip
  inflating: ngrok                   


In [9]:
!./ngrok config add-authtoken 2q9gHoz4I2P9TK9966QZ5snqfJ7_6gb71suopfXZf1Q3cENHe

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
!pip install shortuuid # Install the missing 'shortuuid' module.

In [2]:
!fuser -k 8001/tcp

In [3]:
!pkill -f ngrok

In [4]:
import nest_asyncio
import threading
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.responses import RedirectResponse
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, String, Integer
from sqlalchemy.orm import sessionmaker, declarative_base
import shortuuid
from pyngrok import ngrok
import time
import requests

# Patch asyncio event loop (for Colab or Jupyter)
nest_asyncio.apply()

# FastAPI app
app = FastAPI()

# Database setup
DATABASE_URL = "sqlite:///./test.db"  # using SQLite for local/Colab
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class URL(Base):
    __tablename__ = "urls"
    id = Column(Integer, primary_key=True, index=True)
    original_url = Column(String, nullable=False)
    short_code = Column(String, unique=True, index=True)

Base.metadata.create_all(bind=engine)

# Pydantic schema
class URLRequest(BaseModel):
    original_url: str

# Placeholder for the public ngrok URL
public_url = ""

@app.post("/shorten")
def shorten_url(url_request: URLRequest):
    db = SessionLocal()
    short_code = shortuuid.ShortUUID().random(length=6)
    db_url = URL(original_url=url_request.original_url, short_code=short_code)
    db.add(db_url)
    db.commit()
    db.refresh(db_url)
    db.close()
    return {"short_url": f"{public_url}/{short_code}"}

@app.get("/{short_code}")
def redirect_url(short_code: str):
    db = SessionLocal()
    url = db.query(URL).filter(URL.short_code == short_code).first()
    db.close()
    if url:
        return RedirectResponse(url.original_url)
    raise HTTPException(status_code=404, detail="URL not found")

# Run FastAPI in background thread
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8001)

server_thread = threading.Thread(target=run_server)
server_thread.start()

# Wait a bit for server to be ready
time.sleep(3)

# Create Ngrok tunnel
ngrok_tunnel = ngrok.connect(8001)
public_url = ngrok_tunnel.public_url  # Set the global variable
print("Your FastAPI app is live at:", public_url)

# Test POST /shorten
data = {"original_url": "https://www.wikipedia.org/"}
response = requests.post(f"{public_url}/shorten", json=data)

print("\nResponse status:", response.status_code)
print("Shortened URL:", response.json().get("short_url"))


INFO:     Started server process [11883]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


Your FastAPI app is live at: https://fd09-34-171-216-131.ngrok-free.app
INFO:     34.171.216.131:0 - "POST /shorten HTTP/1.1" 200 OK

Response status: 200
Shortened URL: https://fd09-34-171-216-131.ngrok-free.app/ktswmL
